In [2]:
# Convert ResNet50 to matrix + layerlist and back
import tensorflow as tf
from graph_to_keras import out_from, parse_layer, get_type, Node
from adjacency import to_adjacency_matrix
import numpy as np

keras_model = tf.keras.applications.resnet.ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
# model = tf.keras.applications.NASNetMobile(classes=1000)

#Convert to adjacency matrix + layer_indexes
matrix, layer_indexes = to_adjacency_matrix(keras_model)
matrix = (np.triu(matrix)).T
layers = np.array(layer_indexes)[:,2]

nodelist = []
for layer in layers:
    cfg = parse_layer(layer)
    layer_type = get_type(layer)
    node = Node(cfg, layer_type)
    nodelist.append(node)

for i, row in enumerate(matrix):
    for j, v in enumerate(row):
        if int(v) == 1:
            nodelist[i].inputs.append(nodelist[j])

outputs = out_from(nodelist[-1])
inputs = out_from(nodelist[0])
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(1, 224, 224, 3)]   0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (1, 230, 230, 3)     0           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (1, 112, 112, 64)    9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (1, 112, 112, 64)    256         conv2d[0][0]                     
______________________________________________________________________________________________

In [3]:
keras_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________